In [37]:
import itertools
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit.circuit.library import DraperQFTAdder

In [38]:
a = 1
b = 1


In [ ]:
NUMBITS = 3
NUMQUBITS = NUMBITS * 2
# write the bit representation of a in a_bit inversely
a_bit = [int(i) for i in bin(a)[2:].zfill(NUMBITS)][::-1]
b_bit = [int(i) for i in bin(b)[2:].zfill(NUMBITS)][::-1]

backend = AerSimulator()
adder = DraperQFTAdder(NUMBITS).decompose()

qc = QuantumCircuit(NUMQUBITS)

l = a_bit + b_bit

for i, bit in enumerate(l):
    if bit == 1: 
        qc.x(i)
qc.append(adder, range(NUMBITS * 2))
qc = qc.reverse_bits()
qc.measure_all()
job = backend.run(transpile(qc, backend), shots=1024)
# print(f"Input: {l}. Result: {job.result().get_counts()}")
result = job.result().get_counts()
result = max(result, key=result.get)

result = [int(i) for i in result]
result = result[-NUMBITS:]
# invert the bit order
result = result[::-1]

result = sum([bit * (2 ** i) for i, bit in enumerate(result[::-1])])
print(f"Result: {result}")

Result: 2


In [40]:
qc.draw()

┌─────────────────┐ ░ ┌─┐               
   q_0: ─────┤5                ├─░─┤M├───────────────
             │                 │ ░ └╥┘┌─┐            
   q_1: ─────┤4                ├─░──╫─┤M├────────────
        ┌───┐│                 │ ░  ║ └╥┘┌─┐         
   q_2: ┤ X ├┤3                ├─░──╫──╫─┤M├─────────
        └───┘│  DraperQFTAdder │ ░  ║  ║ └╥┘┌─┐      
   q_3: ─────┤2                ├─░──╫──╫──╫─┤M├──────
             │                 │ ░  ║  ║  ║ └╥┘┌─┐   
   q_4: ─────┤1                ├─░──╫──╫──╫──╫─┤M├───
        ┌───┐│                 │ ░  ║  ║  ║  ║ └╥┘┌─┐
   q_5: ┤ X ├┤0                ├─░──╫──╫──╫──╫──╫─┤M├
        └───┘└─────────────────┘ ░  ║  ║  ║  ║  ║ └╥┘
meas: 6/════════════════════════════╩══╩══╩══╩══╩══╩═
                                    0  1  2  3  4  5